# This notebook fine-tunes the replicated ViT model on CIFAR-10 dataset

In [ ]:
import torch
import torch.nn as nn
from torchvision.models.vision_transformer import ViT_B_16_Weights
import torch.backends.mps

from consts import *
import data
import eval
import vit
import train
import utils

In [ ]:
device = utils.get_device()

In [ ]:
print("Loading CIFAR-10 dataset")
train_ds, train_dl = data.load_CIFAR(
    train=True,
    batch_size=BATCH,
    transforms=ViT_B_16_Weights.DEFAULT.transforms(),
    seed=SEED,
)
test_ds, test_dl = data.load_CIFAR(
    train=False,
    batch_size=BATCH,
    transforms=ViT_B_16_Weights.DEFAULT.transforms(),
    seed=SEED,
)

In [ ]:
classes = test_ds.classes

In [ ]:
utils.show(test_ds, 100)

# Fine-tuning the replicated model

In [ ]:
print("Creating the replicated model")
torch.manual_seed(SEED)
m = vit.ViT(D, IMAGE_W, PATCH, HEADS, DMLP, L, IMAGENET_CLASSES_N, DROPOUT, NORM_EPS)

In [ ]:
print("Loading last saved ViT model")
m = utils.load_last_model(m)

In [ ]:
print("Freezing the model and swapping it's classification head")
# freeze the model
for p in m.parameters():
    p.requires_grad = False

# swap the classification layer
torch.manual_seed(SEED)
m.head = nn.Linear(in_features=D, out_features=len(classes))
m.to(device)

In [ ]:
optim = torch.optim.SGD(m.parameters(), lr=LR, momentum=MOMENTUM)

In [ ]:
print("Fine-tuning the replicated model")
torch.manual_seed(SEED)
train_metrics, test_metrics = train.train(
    m,
    EPOCHS,
    train_dl,
    test_dl,
    device,
    len(classes),
    optim,
    nn.CrossEntropyLoss(),
)

In [ ]:
eval.plot_metrics(train_metrics, test_metrics)

In [ ]:
accuracy = test_metrics[eval.Metrics.ACCURACY.value][-1].item()
utils.save_model(m, accuracy, "FT_CIFAR")